 * Copyright 2024 Xue_Lexiang
 * Licensed under MIT (https://github.com/xuelx1/LearnQC/LISENCE)

## Grover's Algorithm. 

### 1.Initialize and Get Data

In [1]:
from qiskit import QuantumCircuit, transpile
import math
from qiskit_aer import AerSimulator

In [2]:
n = 10
target = ['0000100111', '0000101011']
nx0 = ['0000000000']


### 2.Map the circuit

#### grover iteration: $$ G = -H^n Z_0 H^n Z_f$$

In [3]:
# build Zf and Z0

def add_oracle(qc, n, target_str):
    for i in range(n):
        if target_str[i] == '0':
            qc.x(i)
    qc.mcx(list(range(n)), n)
    for i in range(n):
        if target_str[i] == '0':
            qc.x(i)
    return 0

def build_Zf(qc, n, target):
    qc.x(n)
    qc.h(n)
    for target_str in target:
        add_oracle(qc, n, target_str)
    qc.h(n)
    qc.x(n)
    return 0

# build Grover Iteration

def build_grover_iteration(qc, n, target, nx0):
    build_Zf(qc, n, target)
    qc.h(list(range(n)))
    build_Zf(qc, n, nx0)
    qc.h(list(range(n)))
    return 0

#### repeat Grover Iterations

In [4]:
qc = QuantumCircuit(n+1)
qc.h(list(range(n)))
times = math.floor(math.pi/4*math.sqrt(2**n))
for i in range(times):
    build_grover_iteration(qc, n, target, nx0)
qc.measure_all()

### Execute the Grover algorithm

In [7]:
backend = AerSimulator()
job = backend.run(transpile(qc, backend), shots=1024)
measure = job.result().get_counts()
print(measure)

{'01001001011': 2, '00111011001': 1, '01111001011': 3, '01011000011': 1, '01010111101': 2, '00100000101': 1, '01110010000': 312, '00101110000': 1, '00010111101': 1, '00001010001': 1, '01000110001': 1, '01001011100': 3, '01110111100': 1, '01111010001': 1, '01011001010': 1, '01101001010': 1, '00100011000': 1, '00001111001': 1, '00111101101': 1, '00110101011': 1, '01111010111': 1, '01000110111': 1, '01010110000': 1, '01101010000': 290, '01000100101': 2, '00010111110': 1, '00001010000': 1, '00101110001': 2, '01011011010': 2, '01100111010': 1, '01010000000': 1, '01101101110': 1, '00001100000': 2, '00001111010': 1, '01110010110': 1, '01010101101': 2, '01101101101': 2, '00100011010': 2, '00100000010': 3, '00110100001': 1, '01110110110': 1, '00101001101': 1, '00010100000': 2, '01010110010': 1, '01011001110': 2, '00011000110': 1, '00011101101': 1, '00111010010': 1, '01100111001': 2, '01011011001': 1, '00001001001': 1, '01101011111': 1, '01101000001': 3, '01011000001': 1, '00001000000': 1, '0000

In [10]:
results = {k: v for k, v in measure.items() if v > 10}
print("results: ", results)
total = 0
for key, value in results.items():
    total += value
accuracy = total / 1024
print("accuracy: ", accuracy)

results:  {'01110010000': 312, '01101010000': 290}
accuracy:  0.587890625
